NOTE : Considering some of GEE's user limits and avoiding some system errors, such as "ERROR: user memory limit exceeded","TimeoutError", etc,we will process or download some data beforehand and upload it to GEE Assets, where it will be called up when needed.

Special reminder: the following variables or parameters need to be modified

Must be amended : "roi","region","elevation"

May have to be modified (Depending on your actual situation) : "number_land","number_water","number_seasonal_water"

In [ ]:
#### If you are trying to use geemap in coutries where Gooogle Services are blocked (e.g., China), 
#### you will need a VPN,then replace "10809" with your "proxy port number"to connect to Earth Engine servers.
#### Otherwise, you might encounter a connection timeout issue.

import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [ ]:
#### Initializing GEE

import geemap
import ee
Map=geemap.Map()
Map

In [ ]:
#### Setting the boundaries of the study area.
#### Format: ee.Geometry.Rectangle(minLng, minLat, maxLng, maxLat)

roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
#### Defining variables
region = 'wuhan'

#### Calls for elevation data, which he has downloaded to GEE's Assets in "00--preparation.ipynb"
elevation = ee.Image("users/311605001111/hillshade_wuhan")

#### Visualisation parameters for Landsat8 images
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

# Define the relevant functions

In [ ]:
## Masked invalid pixels

def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

## Calculation of the relevant index
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('(B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7)/10000',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi)

# Define the sample selection function

In [ ]:
#### Set the number of samples
## The number of samples can be adjusted according to the actual situation

number_land = 250
number_water = 3500
number_seasonal_water = 4000

In [ ]:
def imageSample(image):
    ## permanent water
    permanent_points = image.updateMask(permanentWaterExtent).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number_water,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    water_points = permanent_points.map(lambda i : i.setMulti({'waterclass':1,'point_type':'permanent_water','Image_id':image.get('system:id')}))
    ## land
    land_points = image.updateMask(landExtent).sample(**{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': number_land,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    nowater_points = land_points.map(lambda i : i.setMulti({'waterclass':0,'point_type':'no_water','Image_id':image.get('system:id')}))
    sample_points = ee.FeatureCollection([water_points,nowater_points]).flatten()
    return sample_points

def season(image):
    season_points = image.updateMask(seasonWaterExtent_min).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number_seasonal_water,
        'seed': 0,
        'geometries': True,
        'tileScale': 1,
    })
    water_points = season_points.map(lambda i : i.setMulti({'point_type':'season','Image_id':image.get('system:id')}))
    return water_points

# Produce samples at five-year intervals

## permanent water and land samples

You can view the download progress in "Google earth engine->Code Editor->Tasks"

In [ ]:
for year in range(1999,2021,5):
    print(year)
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)  
    l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)           
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
           .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)
    sample_image = ee.ImageCollection(l8.merge(l7).merge(l5)).map(water_index).map(maskSR)
    print('Total number of images：',sample_image.size().getInfo())
    
    # # Determination of the range of waterbodies in each category based on multi-water occurrence products
    JRC_id = 'JRC/GSW1_3/YearlyHistory/' + str(year)
    Maryland_id = 'users/311605001111/Maryland/Maryland_nationwide_' + str(year)
    basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
    JRC = ee.Image(JRC_id).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
    JRC_waterclass = ee.ImageCollection([JRC,basemap]).sum()
    Maryland = ee.Image(Maryland_id).clip(roi).select('b1').rename('waterclass')
    Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
    Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
    Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
    permanentWaterExtent = ee.ImageCollection([JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(2)
    landExtent = ee.ImageCollection([JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(2)
    seasonWaterExtent_min = ee.ImageCollection([JRC_waterclass.eq(1),Maryland_waterclass.eq(1)]).sum().eq(2)
    seasonWaterExtent_max = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')
    #### sampling
    ## "dataset_id" indicates the file name; "assetID" indicates the file path in GEE's Assets.
    ## They need to be modified to suit your situation
    dataset_id = region + '_PWL_' + str(year)
    assetID = 'users/311605001111/' + region + '/' + dataset_id

    points_collection = sample_image.map(imageSample).flatten()    
    task = ee.batch.Export.table.toAsset(**{
        'collection': points_collection,
        'description': dataset_id,
        'assetId': assetID
    })
    task.start()


## seasonal water samples

You can view the download progress in "Google earth engine->Code Editor->Tasks"

In [ ]:
for year in range(1999,2021,5):
    print(year)
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)  
    l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)           
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
           .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)
    sample_image = ee.ImageCollection(l8.merge(l7).merge(l5)).map(water_index).map(maskSR)
    print('Total number of images：',sample_image.size().getInfo())
    
    # # Determination of the range of waterbodies in each category based on multi-water occurrence products
    JRC_id = 'JRC/GSW1_3/YearlyHistory/' + str(year)
    Maryland_id = 'users/311605001111/Maryland/Maryland_nationwide_' + str(year)
    basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
    JRC = ee.Image(JRC_id).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
    JRC_waterclass = ee.ImageCollection([JRC,basemap]).sum()
    Maryland = ee.Image(Maryland_id).clip(roi).select('b1').rename('waterclass')
    Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
    Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
    Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
    permanentWaterExtent = ee.ImageCollection([JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(2)
    landExtent = ee.ImageCollection([JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(2)
    seasonWaterExtent_min = ee.ImageCollection([JRC_waterclass.eq(1),Maryland_waterclass.eq(1)]).sum().eq(2)
    seasonWaterExtent_max = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')
    
    #### sampling
    ## "dataset_id" indicates the file name; "assetID" indicates the file path in GEE's Assets.
    ## They need to be modified to suit your situation
    dataset_id = region + '_SW_' + str(year)
    assetID = 'users/311605001111/' + region + '/' + dataset_id
    
    points_collection = sample_image.map(season).flatten()
    task = ee.batch.Export.table.toAsset(**{
        'collection': points_collection,
        'description': dataset_id,
        'assetId': assetID
    })
    task.start() 

# Combining samples produced in the previous step

Restarting the kernel. Click "Kernel -> Restart & Clear Output" in jupyter notebook

"Define the relevant functions","Define the sample selection function","Produce samples at five-year intervals" etc.They do not need to be executed again, only the remainder is executed

"pw" and "sww" represent the sample data obtained in step 3, which are stored in GEE's Assets

In [ ]:
#### Removing outliers in sample sets 

bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
def removal_outliers(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

In [ ]:
totalSamples = ee.FeatureCollection([])
for year in range(1999,2021,5):
    print(year)
    pw_nw = ee.FeatureCollection('users/311605001111/' + region + '/' + region + '_PWL_' + str(year))
    permanent_water = pw_nw.filter(ee.Filter.eq('waterclass',1)).randomColumn('random',1,'uniform').sort('random').limit(500)
    no_water = pw_nw.filter(ee.Filter.eq('waterclass',0)).randomColumn('random',1,'uniform').sort('random').limit(3000)
    # print('Number of permanent water sampled： ',permanent_water.size().getInfo())
    # print('Number of land sampled： ',no_water.size().getInfo())
    sww = ee.FeatureCollection('users/311605001111/' + region + '/' + region + '_SW_' + str(year))
    sw = sww.randomColumn('random',1,'uniform').sort('random').limit(500)
    # print('Number of seasonal water sampled： ',sw.size().getInfo())
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    cluster_samples = ee.FeatureCollection([permanent_water,no_water]).flatten()
    clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
    q = ee.FeatureCollection(sw.cluster(clusterer))
    q1 = q.filter(ee.Filter.eq('cluster',1))
    q0 = q.filter(ee.Filter.eq('cluster',0))
    b1 = ee.FeatureCollection(q1).filter(ee.Filter.lt('AWEI',0)).size()
    b2 = ee.FeatureCollection(q1).filter(ee.Filter.gt('AWEI',0)).size()
    c1 = ee.FeatureCollection(q0).filter(ee.Filter.lt('AWEI',0)).size()
    c2 = ee.FeatureCollection(q0).filter(ee.Filter.gt('AWEI',0)).size()
    qq1 = ee.Algorithms.If(ee.Number(b1).lt(ee.Number(b2)),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
    qq0 = ee.Algorithms.If(ee.Number(c1).lt(ee.Number(c2)),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
    seasonWater = ee.FeatureCollection([qq1,qq0]).flatten()
    total_Samples = ee.FeatureCollection([permanent_water,no_water,seasonWater]).flatten()
    # print("Number of remaining samples before filtering: ", total_Samples.size().getInfo())
    filtered_samples = removal_outliers(total_Samples)
    # print("Number of remaining samples after filtering: ",filtered_samples.size().getInfo())
    totalSamples = totalSamples.merge(filtered_samples)
    print("Total sample： ",totalSamples.size().getInfo())

In [ ]:
#### The data is stored in GEE's Assets
## "dataset_id" indicates the file name; "assetID" indicates the file path in GEE's Assets.
## They need to be modified to suit your situation

dataset_id = region + '_9920'
assetID = 'users/311605001111/' + region + '/' + dataset_id
task = ee.batch.Export.table.toAsset(**{
    'collection': totalSamples,
    'description': dataset_id,
    'assetId': assetID
})
task.start()

You can view the download progress in "Google earth engine->Code Editor->Tasks"

# Appendix--download

By default, all processing results are saved to GEE's Assets. 

If required, they can also be downloaded to a local disk or Google Drive

## Download to Google Drive

In [ ]:
#### Calling the samples data stored in GEE's Assets. 
## "data_id" indicates the samples data id stored in GEE's Assets

data_id = 'users/311605001111/wuhan/wuhan_9920'
samples = ee.FeatureCollection(data_id)

In [ ]:
#### The data will be stored in local dish
## "dataset_id" indicates the file name; "folder" indicates the file path in local dish.
## They need to be modified to suit your situation

year = 9920
region = 'wuhan'
dataset_id = region + '_' + str(year)
folder = region + '_samples'

task = ee.batch.Export.table.toDrive(**{
    'collection': samples,
    'description':dataset_id,
    'folder': folder,
    'fileFormat': 'shp',
})
task.start()

## Download to local disk

In [ ]:
#### Calling the samples data stored in GEE's Assets. 
## "data_id" indicates the samples data id stored in GEE's Assets

data_id = 'users/311605001111/wuhan/wuhan_9920'
samples = ee.FeatureCollection(data_id)

In [ ]:
#### The data will be stored in local dish
## "folder indicates the file path in local dish.
## For example, the path for this experiment is "D:/wuhan_samples/wuhan_9920.shp"
## They need to be modified to suit your situation

region = 'wuhan'
folder = 'D:/' + region + '_samples/' + region + '_9920.shp'

task = geemap.ee_export_vector(samples, filename= folder)
task.start()